In [ ]:
import pandas as pd

df = pd.read_csv('train-data.dat', sep=',', header=None)
df.columns = ['sentence']
df['sentence'] = df['sentence'].str.replace('<.*?>', '', regex=True)  # deletes <int>
# print(df.head())
df = df[0:1000]
df.head()

vocab = pd.read_csv('vocabs.txt', sep=',', header=None)
vocab.columns = ['word', 'id']

sentences = pd.read_csv('train-data.dat', sep=',', header=None)
sentences.head()

com = []
for line in df.sentence:
    sen = ''
    for word in line.split():
      for id in vocab.id:
        if int(word) == int(id):
          sen = sen + ' ' + str(vocab.word[id]) 
    # print(sen)
    com.append(sen)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_vectors = tfidf_vectorizer.fit_transform(com)

tfidf = tfidf_vectors.todense()
# TFIDF of words not in the doc will be 0, so replace them with nan
tfidf[tfidf == 0] = np.nan
# Use nanmean of numpy which will ignore nan while calculating the mean
means = np.nanmean(tfidf, axis=0)
# convert it into a dictionary for later lookup
means = dict(zip(tfidf_vectorizer.get_feature_names(), means.tolist()[0]))

tfidf = tfidf_vectors.todense()
# Argsort the full TFIDF dense vector
ordered = np.argsort(tfidf*-1)
words = tfidf_vectorizer.get_feature_names()

# top_k = 5
# for i, doc in enumerate(com):
#     result = {}
#     # Pick top_k from each argsorted matrix for each doc
#     for t in range(top_k):
#         # Pick the top k word, find its average tfidf from the
#         # precomputed dictionary using nanmean and save it to later use
#         result[words[ordered[i,t]]] = means[words[ordered[i,t]]]
#     # print (result)

# means

# from collections import Counter 

# means = dict(Counter(means).most_common(1000))


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
li = [[]]

for key in means:
  li.append([key, means[key]])

li.pop(0)
li[:5]

df2 = pd.DataFrame(li)
# df2 = df2.transpose()
df2.head()

tfidf_sc = df2[1]
print(tfidf_sc)

tfidf_len = len(tfidf_sc)
print(tfidf_len)

0       0.180064
1       0.105520
2       0.098096
3       0.163385
4       0.138688
          ...   
6848    0.110620
6849    0.151121
6850    0.124428
6851    0.184492
6852    0.223934
Name: 1, Length: 6853, dtype: float64
6853


In [ ]:
# !pip install deap
import importlib.util

package_name = 'deap'

spec = importlib.util.find_spec(package_name)
if spec is None:
  !pip install deap
else:
  pass

import random
import matplotlib.pyplot as plt
from deap import base
from deap import creator
from deap import tools

POP_SIZE = 20
P_CROSSOVER = 0.6
P_MUTATION = 0.01
MAX_GENERATIONS = 200
GENERATIONS_EXIT = 20  # state the number of generations in which maxFitness remains the same,  for stopping condition


creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 160 kB 5.3 MB/s 


In [ ]:
df2.importance

0       0.180064
1       0.105520
2       0.098096
3       0.163385
4       0.138688
          ...   
6848    0.110620
6849    0.151121
6850    0.124428
6851    0.184492
6852    0.223934
Name: importance, Length: 6853, dtype: float64

In [370]:
# tfidf_sc
df2.columns=['word', 'importance']
# print(df2.head(5))

def evaluation_function(individual):  

  sum = 0
  a = 0
  for i in individual:
    if i==1:
      a += df2.importance[i]
      sum += 1

  counter = 0
  for i in individual:
    if i==1:
      counter+=1
  if counter<1000:
    a -= 50
  else:
    pass
  
  return a, 

In [ ]:
ind = tbx.individual()
pop = tbx.population(5)
evaluation = tbx.evaluate
                                          #TESTING
for i in pop:
  print(evaluation(i)[0])

359.93005413360254
364.88951251656425
361.82942117388575
364.7839921254374
371.00969520192126


In [371]:
tbx = base.Toolbox()

INDIVIDUAL_SIZE = tfidf_len

tbx.register("attr_bool", random.randint, 0, 1)
tbx.register('individual', tools.initRepeat, creator.Individual, tbx.attr_bool, INDIVIDUAL_SIZE)
tbx.register("population", tools.initRepeat, list, tbx.individual)

tbx.register("evaluate", evaluation_function)
tbx.register("select", tools.selTournament, tournsize=3)
tbx.register("mate", tools.cxUniform, indpb=0.5)
tbx.register("mutate", tools.mutFlipBit, indpb=0.05)
# tbx.register("mutate", tools.mutUniformInt, low=0, up=100, indpb=0.8)

In [372]:
pop = tbx.population(POP_SIZE)
len(pop)

20

In [378]:
fitnesses = list(map(tbx.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
  ind.fitness = fit
  print(ind.fitness)

(358.03068709331933,)
(367.8440834681159,)
(356.7644423997972,)
(368.37168542375014,)
(357.7141259199388,)
(371.3262563753018,)
(360.87973765374414,)
(367.6330426858622,)
(360.985258044871,)
(357.5030851376851,)
(360.2466153069831,)
(361.40733960937837,)
(358.98037061346093,)
(359.4024521779683,)
(372.0648991131897,)
(360.985258044871,)
(368.37168542375014,)
(355.49819770627505,)
(353.493310274865,)
(363.51774743191527,)


In [481]:
CXPB, MUTPB = 0.5, 0.2

fits = [ind.fitness[0] for ind in pop]

MAX_GENERATIONS = 10
# Variable keeping track of the number of generations
g = 0

# Begin the evolution
while g < MAX_GENERATIONS:
  # A new generation
  g = g + 1
  print("-- Generation %i --" % g)

  # Select the next generation individuals
  offspring = tbx.select(pop, len(pop))
  # Clone the selected individuals
  offspring = list(map(tbx.clone, offspring))

  # Apply crossover and mutation on the offspring
  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if random.random() < CXPB:
      tbx.mate(child1, child2)
      del child1.fitness
      del child2.fitness

  for mutant in offspring:
    if random.random() < MUTPB:
      # print('new')
      # print(offspring.index(mutant))
      # print(mutant.fitness)
      try:
        tbx.mutate(mutant)
        del mutant.fitness
        # print('k')
      except:
        pass

  # Evaluate the individuals with an invalid fitness
  invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
  fitnesses = map(tbx.evaluate, invalid_ind)
  for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit

-- Generation 1 --


AttributeError: ignored

In [482]:
# for i in range(len(offspring)):
#   print(i)
#   print(int(offspring[i].fitness[0]))

for i in offspring:
  try:
    print(i.fitness)
  except:
    print(i)

[0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 